In [1]:
# import tensorflow as tf
# tf.compat.v1.enable_eager_execution()
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
le = preprocessing.LabelEncoder()

In [2]:
# Load data
data = pd.read_csv("data/v2/data.csv",  header=None)
target = pd.read_csv("data/v2/id.csv", header=None)

In [3]:
#print(data.head)
print("INFO data")
print(data.info(verbose=True))
print("INFO target")
print(target.info(verbose=True))

INFO data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 128 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       float64
 1   1       float64
 2   2       float64
 3   3       float64
 4   4       float64
 5   5       float64
 6   6       float64
 7   7       float64
 8   8       float64
 9   9       float64
 10  10      float64
 11  11      float64
 12  12      float64
 13  13      float64
 14  14      float64
 15  15      float64
 16  16      float64
 17  17      float64
 18  18      float64
 19  19      float64
 20  20      float64
 21  21      float64
 22  22      float64
 23  23      float64
 24  24      float64
 25  25      float64
 26  26      float64
 27  27      float64
 28  28      float64
 29  29      float64
 30  30      float64
 31  31      float64
 32  32      float64
 33  33      float64
 34  34      float64
 35  35      float64
 36  36      float64
 37  37      float64
 38  38      float64
 39  39      float64
 4

In [4]:
# Data train into numpy
X = np.array(data.values)
print("INFO data train ndim ", X.ndim)
print("INFO data train shape ", X.shape)
print("INFO data train example", X[:-10])
# Data target into numpy
y = np.array(target.values)
le.fit([0, 1, 2, 3, 4, 5, 6, 7])
y = le.transform(y)
y = lb.fit_transform(y)
print("INFO data target ndim ", y.ndim)
print("INFO data target shape ", y.shape)
print("INFO data target example", y[:])

INFO data train ndim  2
INFO data train shape  (403, 128)
INFO data train example [[-0.02098308  0.14330864 -0.06685185 ... -0.08917168 -0.03871956
  -0.02110493]
 [-0.00516151  0.14268468 -0.03437199 ... -0.0891209  -0.01322121
  -0.02910456]
 [-0.00589333  0.09212484 -0.07247945 ... -0.10010736 -0.05481427
  -0.03378614]
 ...
 [-0.08088823  0.06747486  0.12673211 ... -0.07030995  0.0410323
  -0.0464632 ]
 [-0.0644524   0.03922407  0.1310685  ... -0.06172986  0.04847588
  -0.07036125]
 [-0.0860193   0.02960051  0.10438477 ... -0.09377543  0.0331197
  -0.05067415]]
INFO data target ndim  2
INFO data target shape  (403, 8)
INFO data target example [[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


In [5]:
# Separate train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
# import tensorflow keras backend
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, Conv1D, MaxPooling1D

In [7]:
(1,X_train.shape[1])

(1, 128)

In [104]:
# Build keras model template
def build_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=512, activation='tanh', input_dim=(X_train.shape[1])))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Dense(units=256, activation='tanh', input_dim=(X_train.shape[1])))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Dense(units=128, activation='tanh', input_dim=(X_train.shape[1])))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Dense(units=8, activation='softmax', name="output_node"))          
    model.compile(optimizer='sgd',
                loss=tf.keras.losses.cosine_proximity,
                metrics=[tf.keras.metrics.cosine_proximity])
    return model

def create_model(my_learning_rate):
  """Create and compile a deep neural net."""
  
  # All models in this course are sequential.
  model = tf.keras.models.Sequential()

  # The features are stored in a two-dimensional 28X28 array. 
  # Flatten that two-dimensional array into a a one-dimensional 
  # 784-element array.

  # Define the first hidden layer.   
  model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=(X_train.shape[1])))
  
  # Define a dropout regularization layer. 
  model.add(tf.keras.layers.Dropout(rate=0.2))

  # Define the output layer. The units parameter is set to 10 because
  # the model must choose among 10 possible output values (representing
  # the digits from 0 to 9, inclusive).
  #
  # Don't change this layer.
  model.add(tf.keras.layers.Dense(units=8, activation='softmax', name="output_node"))    
                           
  # Construct the layers into a model that TensorFlow can execute.  
  # Notice that the loss function for multi-class classification
  # is different than the loss function for binary classification.  
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=[tf.keras.metrics.categorical_crossentropy])
  
  return model  

In [105]:
# Building model
model = build_model()
#model = create_model(0.0005)

In [52]:
ts = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
ts

'2020-09-25 12:08:35'

In [53]:
# Compile the model version 1

cp_callback = tf.keras.callbacks.ModelCheckpoint("model/ckps/v1/weights.{epoch:02d}-v1.hdf5", save_weights_only=True, verbose=0)

In [54]:
# Compile the model version 2
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0003), 
              loss=tf.keras.losses.categorical_crossentropy, 
              metrics=['accuracy'])
cp_callback = tf.keras.callbacks.ModelCheckpoint("model/ckps/v2/weights.{epoch:02d}-v2", save_weights_only=True, verbose=0)

In [106]:
# Train model
 model.fit(X_train, y_train, callbacks=[cp_callback], epochs=50)
#model.fit(X_train, y_train, epochs=20)

_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
322/322 [==============================] - 0s 104us/step - loss: -0.6551 - cosine_proximity: -0.6551
Epoch 18/50
 32/322 [=>............................] - ETA: 0s - loss: -0.6627 - cosine_proximity: -0.6627WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.SGD object at 0x7fcf80079438>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
322/322 [==============================] - 0s 111us/step - loss: -0.6738 - cosine_proximity: -0.6738
Epoch 19/50
 32/322 [=>............................] - ETA: 0s - loss: -0.6683 - cosine_proximit

In [56]:
# Evaluate model
pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=True)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))  
pred_test  

81/81 [==============================] - 0s 2ms/step
Accuracy on test data: 1.0% 
 Error on test data: 0.0


array([[0.10455891, 0.29369104, 0.08328355, 0.09000523, 0.18341474,
        0.08344128, 0.11073275, 0.05087246],
       [0.10185516, 0.14293091, 0.11493094, 0.1250131 , 0.15363102,
        0.17495827, 0.09761094, 0.08906976],
       [0.09723119, 0.14584431, 0.10361267, 0.12970771, 0.15722628,
        0.187004  , 0.09385962, 0.08551424],
       [0.22772565, 0.14608291, 0.08263063, 0.09691924, 0.11839603,
        0.10907465, 0.1449955 , 0.07417544],
       [0.22880669, 0.15263751, 0.08907725, 0.09064167, 0.1267052 ,
        0.10637036, 0.13904224, 0.06671909],
       [0.11290651, 0.2823962 , 0.08944061, 0.0918517 , 0.17102017,
        0.08813198, 0.11073631, 0.05351654],
       [0.09831645, 0.11207079, 0.2575367 , 0.1157933 , 0.10379592,
        0.0849204 , 0.14875942, 0.07880698],
       [0.08585478, 0.12283859, 0.10187361, 0.13289754, 0.07523537,
        0.08055551, 0.14015415, 0.26059043],
       [0.13149361, 0.09492166, 0.10106439, 0.0720276 , 0.06508715,
        0.07241431, 0.360942

In [107]:
# save model
# tf.saved_model.save(model,"model/1")

# Save entire model to a HDF5 file
model.save('model/h5/model_v2.h5')

model.save('model/saved_model/model_v2.pb')
print('Saved model ...')

Saved model ...


In [58]:
print(model.input.op.name)
print(model.output.op.name)
print(model.summary())

dense_9_input
output_node_4/Softmax
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
output_node (Dense)          (None, 8)                 264       
Total params: 4,392
Trainable params: 4,392
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# Saving graph version 1 model
with tf.Graph().as_default():
  with tf.Session() as sess:
        model = build_model()

        model.compile(optimizer='RMSprop', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
        """cp_callback = tf.keras.callbacks.ModelCheckpoint("model/ckps/weights.{epoch:02d}-v1.hdf5", save_weights_only=True,verbose=0)"""
        model.fit(X_train, y_train, epochs=20)

        # Evaluate model
        pred_test= model.predict(X_test)
        scores2 = model.evaluate(X_test, y_test, verbose=True)
        print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))  
        pred_test

        # Initialize all variables
        init = tf.global_variables_initializer()
        sess.run(init)
        saver = tf.train.Saver()
        saver.save(sess,'model/graph/model_v1.ckpt')
        tf.train.write_graph(sess.graph.as_graph_def(), 'model/graph/v1', 'model_v1_.pbtxt', as_text=True) 

In [108]:
# Saving graph version 2 model
with tf.Graph().as_default():
  with tf.Session() as sess:
        model = build_model()

        model.fit(X_train, y_train, epochs=50)

        # Evaluate model
        pred_test_mog = model.predict(X_test)
        pred_test = np.argmax(pred_test_mog, axis=1)
        pred_test = le.transform(pred_test)
        pred_test = lb.fit_transform(pred_test)

        from sklearn.metrics import accuracy_score
        scores1 = accuracy_score(y_test,pred_test)
        print('[SCORE1] Accuracy on test data: {}% \n Error on test data: {}'.format(scores1, 1 - scores1))

        pred_test

        # Initialize all variables
        init = tf.global_variables_initializer()
        sess.run(init)
        saver = tf.train.Saver()
        saver.save(sess,'model/graph/v2/model_v2.ckpt')
        tf.train.write_graph(sess.graph.as_graph_def(), 'model/graph/v2', 'model_v2_.pbtxt', as_text=True) 

Epoch 1/50
322/322 [==============================] - 0s 514us/step - loss: -0.3631 - cosine_proximity: -0.3631
Epoch 2/50
322/322 [==============================] - 0s 110us/step - loss: -0.3746 - cosine_proximity: -0.3746
Epoch 3/50
322/322 [==============================] - 0s 89us/step - loss: -0.3910 - cosine_proximity: -0.3910
Epoch 4/50
322/322 [==============================] - 0s 103us/step - loss: -0.4026 - cosine_proximity: -0.4026
Epoch 5/50
322/322 [==============================] - 0s 91us/step - loss: -0.4144 - cosine_proximity: -0.4144
Epoch 6/50
322/322 [==============================] - 0s 91us/step - loss: -0.4332 - cosine_proximity: -0.4332
Epoch 7/50
322/322 [==============================] - 0s 86us/step - loss: -0.4508 - cosine_proximity: -0.4508
Epoch 8/50
322/322 [==============================] - 0s 94us/step - loss: -0.4694 - cosine_proximity: -0.4694
Epoch 9/50
322/322 [==============================] - 0s 89us/step - loss: -0.4830 - cosine_proximity: -0.483

In [109]:
# Graph with version 1
from tensorflow.python.framework import graph_io

tf.keras.backend.clear_session()

def freeze_graph(graph, session, output, save_pb_dir='.', save_pb_name='model/graph/saved/frozen_model.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen

tf.keras.backend.set_learning_phase(0) 

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=(X_train.shape[1])))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax', name="output_node"))

model.compile(optimizer='RMSprop', 
        loss='categorical_crossentropy', 
        metrics=['accuracy'])
"""cp_callback = tf.keras.callbacks.ModelCheckpoint("model/ckps/weights.{epoch:02d}-v1.hdf5", save_weights_only=True,verbose=0)"""
model.fit(X_train, y_train, epochs=5)

# Evaluate model
pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=True)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))  
pred_test

session = tf.keras.backend.get_session()

input_names = [t.op.name for t in model.inputs]
output_names = [t.op.name for t in model.outputs]

# Prints input and output nodes names, take notes of them.
print(input_names, output_names)

frozen_graph = freeze_graph(session.graph, session, [out.op.name for out in model.outputs])

ValueError: Error when checking target: expected output_node to have shape (6,) but got array with shape (8,)

In [112]:
# Graph with version 2
from tensorflow.python.framework import graph_io
import tensorflow as tf

tf.keras.backend.clear_session()

def freeze_graph(graph, session, output, save_pb_dir='.', save_pb_name='model/graph/v2/saved/frozen_model.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen

tf.keras.backend.set_learning_phase(0) 
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=512, activation='tanh', input_dim=(X_train.shape[1])))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=256, activation='tanh', input_dim=(X_train.shape[1])))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=128, activation='tanh', input_dim=(X_train.shape[1])))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=8, activation='softmax', name="output_node"))          
model.compile(optimizer='sgd',
            loss=tf.keras.losses.cosine_proximity,
            metrics=[tf.keras.metrics.cosine_proximity])

model.fit(X_train, y_train, epochs=50)

# Evaluate model
pred_test_mog = model.predict(X_test)
pred_test = np.argmax(pred_test_mog, axis=1)
pred_test = le.transform(pred_test)
pred_test = lb.fit_transform(pred_test)

from sklearn.metrics import accuracy_score
scores1 = accuracy_score(y_test,pred_test)
print('[SCORE1] Accuracy on test data: {}% \n Error on test data: {}'.format(scores1, 1 - scores1))

session = tf.keras.backend.get_session()

input_names = [t.op.name for t in model.inputs]
output_names = [t.op.name for t in model.outputs]

# Prints input and output nodes names, take notes of them.
print(input_names, output_names)

frozen_graph = freeze_graph(session.graph, session, [out.op.name for out in model.outputs])

Epoch 1/50
322/322 [==============================] - 0s 405us/step - loss: -0.3509 - cosine_proximity: -0.3509
Epoch 2/50
322/322 [==============================] - 0s 68us/step - loss: -0.3635 - cosine_proximity: -0.3635
Epoch 3/50
322/322 [==============================] - 0s 90us/step - loss: -0.3768 - cosine_proximity: -0.3768
Epoch 4/50
322/322 [==============================] - 0s 81us/step - loss: -0.3908 - cosine_proximity: -0.3908
Epoch 5/50
322/322 [==============================] - 0s 126us/step - loss: -0.4052 - cosine_proximity: -0.4052
Epoch 6/50
322/322 [==============================] - 0s 97us/step - loss: -0.4200 - cosine_proximity: -0.4200
Epoch 7/50
322/322 [==============================] - 0s 81us/step - loss: -0.4351 - cosine_proximity: -0.4351
Epoch 8/50
322/322 [==============================] - 0s 85us/step - loss: -0.4507 - cosine_proximity: -0.4507
Epoch 9/50
322/322 [==============================] - 0s 77us/step - loss: -0.4678 - cosine_proximity: -0.4678

In [111]:
from tensorflow.python.tools import freeze_graph

freeze_graph.freeze_graph('model/graph/v2/model_v2_.pbtxt', "", False, 
                          'model/graph/v2/model_v2.ckpt', model.output.op.name,
                           "save/restore_all", "save/Const:0",
                           'model/graph/v2/saved/model_v2_.pb', True, "")

Instructions for updating:
Use tf.gfile.GFile.


IndexError: list index out of range